In [ ]:

import IPython
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np 
import numpy.random as npr
import scipy as sp
import scipy.linalg as sl
from scipy.fft import fft
from scipy.fft import ifft
plt.rcParams["animation.html"]="jshtml"


**Introduction**

Dans ce rapport nous essaierons de refaire l'experience des fentes de Young au travers d'un simulateur de Schrodinger
Nous commencerons d'abord par tester le simulateur lors de divers probleme 1D, voir si les resultats sont bien coherents avec les resultats historiques et theoriques

Puis nous regarderons des simulations de probleme en 2D voir si nous avons les memes resultats ou des resultats semblables

Enfin nous regarderons les resultats pour l'experience des fentes de Young

Tout d'abord, nous allons regarder la boite a outils mise a notre disposition pour ce rapport. En dehors du simulateur de l'equation de schrodinger par methode spectral qui a deja ete detailler, nous expliquerons quelques outils qui nous seront utiles lors de la suite de ce rapport.

**Les potentiels:**

Pour verifier l'exactitude du simulateurs, nous regarderons les simulations de l'equation de schrodinger avec plusieurs potentiels differents:

- potentiel nul : cas simple qui nous permet de tester juste la partie avec le laplacien

- barriere de potentiel ou fonction creneau : nous permet de tester l'effet tunnel sur le simulateur

- puit de potentiel : nous permet de tester des etats stationnaire dans un potentiel non nul et où l'on peut rationnaliser la periodicite induite par le simulateur

- potentiel quadratique : nous permet de tester des etats stationnaire dans un potentiel smooth.


In [ ]:
#################  potentiel 1D ###############

V_0 =           lambda x: 0*x
V_barriere =    lambda x,L,V0,d:  (x>d)*(x<(d+L))*V0
V_puit =        lambda x,V0,d:    ((x<-d)+ (x>d))*2*V0
V_quad =        lambda x,x0,V0 :  V0* (x-x0)**2

################  potentiel 2D ################        
   
ray = lambda x,y: np.sqrt(x**2 +y**2)
V_0_2D =           lambda x,y:0*x*y
## potentiels ~2D
V_barriere_2D = lambda x,y,L,V0,d:  (x>d)*(x<(d+L))*V0      +y*0
V_tube_2D =     lambda x,y,L,V0,d:  ((x<d) + (x>(d+L)))*V0  +y*0
V_quad_2D =     lambda x,y,x0,y0,V0 :  V0*(x-x0)**2         +y*0
## potentiels radiaux
V_couronne_2D = lambda x,y,x0,y0,L,V0,d:  V_barriere(ray(x-x0,y-y0),L,V0,d)
V_puit_2D  =    lambda x,y,x0,y0,V0,d:    V_puit(ray(x-x0,y-y0), V0,d)
V_quad_2D =     lambda x,y,x0,y0,V0:      V_quad(ray(x-x0,y-y0), 0,V0)  

**Jauges**

Les "Jauges" seront des outils pour jauger differentes quantites de la simulation et/ou voir  si elles se conservent comme en theorie. Parmis les Jauges, on note :

- la jauge de probabilite global ou norme_psi pour verifier la conservation de la probabilite 

- la jauge de probabilite local ou concentration de masse pour voir les coefficients de reflexion-transmission

- la Jauge d'energie pour regarder la conservation de l'energie

In [ ]:
def Norme_psi_Hs(psi,Nx,L, reduction=1,s=1.):
    Nx = int(Nx/reduction)
    C_psi = fft(psi)
    norm = np.abs(C_psi)**2 * pow((np.fft.fftfreq(Nx,L/Nx)**2 +1),s)
    norm *= reduction**2
    return sum(norm)


def Norme_psi(psi,Nx,L, reduction=1):
    Nx = int(Nx/reduction)
    C_psi = fft(psi)
    norm = np.abs(C_psi)**2 
    norm *= reduction**2
    return sum(norm)



def Concentration_de_masse(psi,Nx,L,a,b,reduction=1):
    Nx = int(Nx/reduction)
    l = np.abs(b-a)
    an = int((np.abs(a+L)/(2*L))*Nx) ; bn = int((np.abs(b+L)/(2*L))*Nx)
    psi_l= np.zeros(Nx, dtype='complex')
    psi_l[an:bn] = psi[an:bn]
    return Norme_psi(psi_l,Nx,L)* reduction**2

def Energie(psi,V,Nx,L,t,reduction=1):
    Nx = int(Nx/reduction)
    I = np.linspace(-L,L,Nx)
    C_psi = fft(psi); V_psi= fft(V(I,t)*psi)
    energ = 0.5*np.abs(C_psi)*(np.fft.fftfreq(Nx,L/Nx)**2) + np.abs(V_psi)
    energ *= reduction**2
    return sum(energ)


def Jauges(psi,Nx,L,t, V_fun=(lambda x,t: 0)):
    print("norme de psi sur [-L,L] = ", Norme_psi(psi,Nx,L))
    print("Energie de psi = ", Energie(psi,V_fun,Nx,L,t))
    print("============================== ",t," ===========================")


**Regularisation des potentiels**

Un probleme que l'on peut vouloir etudie est l'erreur et la dispersion numerique induite par la discontinuite des potentiels. 

Pour regarder leurs effets, nous avons mis au point des potentiels similaire regularise avec des splines $C^\infty$

Les potentiels 2D equivalents aux potentiels sont reguliers car ce sont des potentiels radiaux ou des potentiels 1D auquel on a rajoute une dimension. On peut donc appliquee un spline 1D sans probleme et rajouter de la regularite de la meme maniere que si il s'agissait d'un potentiel 1D.

In [ ]:
def splin_exp(X):
    Nx = X.size
    Y=np.empty_like(X,dtype='complex')
    for i in range(Nx):
        x = X[i]
        if(x<0): Y[i]=0
        elif(x>1): Y[i]= 1
        else :Y[i]= np.exp(-1/x) / (np.exp(-1/x)+ np.exp(-1/(1-x)))
    return Y

## raccorde [(p1,a),(p2,b)]
raccord = lambda x,p1,p2,a,b: p1 + splin_exp((x-a)/(b-a))*(p2-p1)

#  V0*(1-(splin_exp((x-(d+L))/(-epsi))))*(x>d+L)
Vr_barriere =    lambda x,L,V0,d,epsi:  V_barriere(x,L,V0,d) + raccord(x,0,V0,d-epsi,d)*(x<d) + raccord(x,V0,0,d+L,d+L+epsi)*(x>d+L)
Vr_puit =        lambda x,V0,d,epsi:    V_puit(x,V0,d+epsi) +  raccord(x,0,V0,d,d+epsi)*(x<d) + raccord(x,V0,0,-d,-d+epsi)*(x>-d)

################  potentiel 2D ################        
ray = lambda x,y: np.sqrt(x**2 +y**2)
## potentiels ~2D
Vr_barriere_2D = lambda x,y,L,V0,d,epsi:  V_barriere_2D(x,y,L,V0,d)*splin_exp((x-(d-epsi))/(2*epsi))
Vr_tube_2D =     lambda x,y,L,V0,d,epsi:  V_tube_2D(x,y,L,V0,d) *   splin_exp((x-(d-epsi))/(2*epsi))
## potentiels radiaux
Vr_couronne_2D = lambda x,y,x0,y0,L,V0,d,epsi: V_couronne_2D(x,y,x0,y0,L,V0,d)*splin_exp((ray(x-x0,y-y0)-(d-epsi))/(2*epsi)) *splin_exp((ray(x-x0,y-y0)-(d+L-epsi))/(2*epsi))
Vr_puit_2D  =    lambda x,y,x0,y0,L,V0,d,epsi: V_puit_2D(x,y,x0,y0,L,V0,d)    *splin_exp((ray(x-x0,y-y0)-(d-epsi))/(2*epsi))

**Autre fonctions utiles**


- eig_function : etat stationnaire pour le potentiel nul de norme $A$ que l'on peut module par une phase $\phi$

- weyl_func :    etat quasi stationnaire, on peut le considerer comme un element d'une suite de Weyl. Sa fonction de densite est celle d'un etat stationnaire que l'on multiplie par une gaussienne centree en $x_0$ de variance $\sigma$. 

- projection_red et projection_red_2D: deux fonctions avec le meme but, reduire la taille de stockage memoire de la solution. elle reduisent d'un facteur red_x, ( ou red_x*red_y) le poid memoire de la solution.
Cela permet de reduire le poids de la solution sans impacter son erreur

In [ ]:
def eig_func(x,k,A=1.,phi=0+0j):
    return A*np.exp(1j*x*2*np.pi*k)*np.exp(1j*phi)

def weyl_func(x,k,A=1.,phi=0+0j,x0=0.,sig=1.):
    return np.exp(-(x-x0)**2/(sig)**2) * eig_func(x,k,A,phi)

def Projection_red (Nx,psi,red_x):
    sol_red = np.zeros(int(Nx/red_x), dtype="complex")
    for i in range(0,Nx,red_x):
        sol_red[int(i/red_x)] = psi[i]
    return sol_red

def Projection_red_2D (Nx,Ny,psi,red_x=1,red_y=1):
    sol_red = np.zeros((int(Nx/red_x),int(Ny/red_y)), dtype="complex")
    for i in range(0,Nx,red_x):
        for j in range(0,Ny,red_y):
            sol_red[int(i/red_x),int(j/red_y)] = psi[i,j]
    return sol_red

**Quelques commentaire sur le simulateur de schrodinger**

on peut deja voir en argument d'entre, "*reduction*" et "*dilatation_temps*". Ces arguments vont nous servir pour ne pas avoir a afficher ou stocker toute la solution mais seulement une partie.

En pratique, la solution psi ne possede que une coordonnee d'espace tout les *reduction* et elle a ete calcule sur $N_t$ *dilatation_temps* pas de temps. On demandera que *reduction* divise $N_x$ et que *dilatation_temps* soit un entier.


In [ ]:
def dynamics_fft_diss(psi0_fun=(lambda x: np.exp(-x**2)), V_fun=(lambda x,t: 0.), L=10., Nx=100, T=4., Nt=100, reduction=1, show_phi=False, dilatation_temps=1):
    
    T = T/dilatation_temps
    dt = T/Nt; dx = L/Nx
    # Kinetic = Kin(Nx)
    Kinetic = (0.5*(2*np.pi/L)**2) *np.fft.fftfreq(Nx, dx)*np.fft.fftfreq(Nx, dx)
    I = np.linspace(-L, L,Nx,endpoint=False)
    

    Psi_temp =np.zeros((Nx,2), dtype="complex"); Phi_temp = np.zeros((Nx), dtype="complex")
    Psi_temp[:,0]=psi0_fun(I)
    
    Psi_T = np.zeros((int(Nx/reduction),Nt), dtype="complex")

    if(not show_phi) : Psi_T[:,0]= Projection_red(Nx,Psi_temp[:,0],reduction)
  
    for i in range(1,Nt*dilatation_temps):
        ti = dt*i
        Phi_temp[:]= (np.exp(-1j*Kinetic*dt)) * fft(np.exp(-1j*V_fun(I,ti)*dt) * Psi_temp[:,0])
        Psi_temp[:,1]= ifft(Phi_temp[:])
        
        if(i%dilatation_temps==0) : 
            if(not show_phi) :  Psi_T[:,int(i/dilatation_temps)]  = Projection_red(Nx,Psi_temp[:,1],reduction); 
            else :              Psi_T[:,int(i/dilatation_temps)]  = Projection_red(Nx,Phi_temp[:],reduction); 
        
        Psi_temp[:,0] = Psi_temp[:,1]

    return Psi_T

In [ ]:
def plot_psi(psi, duration=10, frames_per_second=20, L=10, show_potential=False,  V_fun=(lambda x,t: 0), Time_length=1, subtitle=' '):
    
    fig, ax = plt.subplots()
    t_data = np.linspace(0, 1, np.size(psi, 1)) # 1 is arbitrary here
    x_data = np.linspace(-L,L,  np.size(psi,0))
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, np.min(np.real(psi)), np.min(np.imag(psi)))
    M = np.max(np.abs(psi))
    
    # set the axis once and for all
    ax.set(xlim=[-L,L], ylim=[m,M], xlabel='x', ylabel='psi')
    # dummy plots, to update during the animation
    real_plot = ax.plot(x_data, np.real(psi[:, 0]), label='Real')[0]
    imag_plot = ax.plot(x_data, np.imag(psi[:, 0]), label='Imag')[0]
    abs_plot  = ax.plot(x_data, np.abs(psi[:, 0]), label='Abs', )[0]
    if(show_potential):
        ax_pot = ax.twinx()
        V_plot  =   ax_pot.plot(x_data, V_fun(x_data,0), 'r', label='V')[0]
        ax_pot.set_ylabel('Potentiel')
        ax_pot.legend(fancybox=True, shadow=True, loc='upper right')
    ax.legend(fancybox=True, shadow=True, loc='upper left')
    fig.suptitle(subtitle)
    

    # define update function as an internal function (that can access the variables defined before)
    # will be called with frame=0...(duration*frames_per_second)-1
    def update(frame):
        IPython.display.clear_output(wait=True)
        print(frame)
        # get the data by linear interpolation
        t = frame / (duration * frames_per_second) 
        psi_t = np.array([np.interp(t, t_data, psi[i, :]) for i in range(np.size(psi,0))])
        # update the plots
        ax.set_title(f"solution en t={ np.floor(100*t*Time_length)/100 }")
        real_plot.set_ydata(np.real(psi_t))
        imag_plot.set_ydata(np.imag(psi_t))
        abs_plot.set_ydata(np.abs(psi_t))
        if(show_potential):
            V_plot.set_ydata(V_fun(x_data,t))

    ani = animation.FuncAnimation(fig=fig, func=update, frames=duration*frames_per_second, interval=1000/frames_per_second)
    return ani


# Probleme 1D
- Pour le potentiel nul, on s'attends a voir que si on raccorde les etat stationnaire pour qu'ils deviennent periodique sur la zone de simulation, alors ils ne devraient etre.. stationnaire.
- On remarque que en plus de conserver la norme L2, il conserve l'energie.

In [ ]:
## parametre simulation
Nx = 1000; Nt=500; L=1
## parametre physique
k=1; T=1

I = np.linspace(-L,L,Nx)

## on normalise la fonction
C=1/np.sqrt(Norme_psi(eig_func(I,L*k,1),Nx,L))
psi0 =lambda x: eig_func(x,L*k,C)
V = lambda x,t: V_0(x) 


psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt)
for i in range(0,Nt,int(np.floor(Nt/10))):
    Jauges(psi[:,i],Nx,L,i/Nt)

In [ ]:

anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T,subtitle="potentiel nul")
plt.close()
anime

**barriere de potentiel**

- En premier lieu, il est important de remarquer que si l'on choisit bien les parametres $k,V_0$ et $l$, on voit apparaitre un phenomene similaire a l'effet tunnel. Cependant, si l'on augmente le parametre $l$, on voit que le phenomene est plus semblable a une reflexion-transmission. Pour eviter d'avoir une partie de la probabilite pieger dans le potentiel et pour negliger la difference entre les deux phenomene, nous allons poser $l<<L$
- on note que si l'on pose $N_t<N_x$ alors on obtiens des phenomene de dispersion numerique lorsque l'onde interagit avec la barriere de potentiel.

- on remarque que l'on conserve la probabilite au cours de la simulation. On ne conserve cependant pas l'energie. Cela est due aux phenomenes de reflexion-transmission induit par le potentiel. 

Cette perte de conservation est encore plus claire lorsque l'on regarde le graphe de l'energie de de la norme $L^2$ au cours du temps. On voit que lorsque la fonction d'onde n'interagit pas avec la barriere, la probabilite et l'energie sont conserve mais lorsqu'il y a interaction, il y a un changement de l'energie. Apres l'interaction, l'energie se stabilise a un niveau legerement plus elevee que precedemment. La difference a l'air d'etre proportionnel a $k^{-1}$ et $V_0$

In [ ]:
## parametre simulation
Nx = 2000; Nt=5000; L=10;    dilatation_temps =4; reduction=1
## parametre physique
k=8+0j; T=10; d=0; l=0.1; V0=50; 

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= 1/ np.sqrt(Norme_psi(weyl_func(I,k=k,A=1,sig=0.5,x0=-3),Nx,L))

psi0 =lambda x: weyl_func(x,k=k,A=C,sig=0.5,x0=-3)
V = lambda x,t: V_barriere(x,l,V0,d) 

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)

### analyse des resultats
for i in range(0,Nt,int(np.floor(Nt/5))):
    t = T*i/Nt
    print("============================== ",t," ===========================")
    print(Concentration_de_masse(psi[:,i],Nx,L,-L,d)," masse avant l'interface")
    print(Concentration_de_masse(psi[:,i],Nx,L,d,d+l)," masse dans l'interface")
    print(Concentration_de_masse(psi[:,i],Nx,L,d+l,L)," masse apres l'interface")


N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt
    N.append(Norme_psi(psi[:,i],Nx,L,reduction=reduction)); E.append( Energie(psi[:,i],V,Nx,L,t,reduction=reduction))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b', label="norme $L^2$")
ax2.plot(time,E,'r', label="Energie")
ax.legend(loc="upper left"); ax2.legend(loc="upper right")
ax.set_xlabel("t")
fig.suptitle("conservation de la norme et de l'energie")
plt.show()

In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="barriere de potentiel")
plt.close()
anime

on constate cependant que les coefficients de reflexion-transmission ne sont pas en accord avec la theorie, on peut calculer les coefficients de transmission reflexion theorique en etudiant la solution globale stationnaire comme $3$ ondes planes sur $3$ milieux differents, devant, dans et derriere la barriere. Apres quelques considerations de regularite, on note que etant solution stationnaire, elle est dans $H^1$ et donc continue dans $R$, on supposera de plus que les derivees sont continues.

Alors, si une onde arrive d'un côte, et repars de l'autre, avec les conditions aux bords de la barriere, on devrait donc trouver un coefficient de transmission semblable a:

\begin{equation}
    T \simeq \frac{4K^2k^2}{4K^2k^2+ (K^2+k^2)^2 + (Kl)^2}  \qquad K=\sqrt{2(V_0 -\frac{k^2}{2})}
\end{equation}
L'approximation vient du terme $(Kl)^2$ au denominateur qui se trouve etre $\sinh(Kl)^2$ que l'on peut approcher par un DL a l'ordre 1 en 0 en supposant que l est aussi negligeable devant $K$.
Il s'agit aussi tres probablement du terme posant probleme car il depend de l'epaisseur de la barriere la où la methode n'en depend pas. 

On peut constater l'independance du taux de transmissions par l'exemple ci-dessous où l'on constate le meme taux de transmission si $l=2$ ou $l=0.2$

In [ ]:
## parametre simulation
Nx = 2000; Nt=5000; L=10
## parametre physique
k=30+0j; T=10; d=0; l=2; V0=20; dilatation_temps =4; reduction=1

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= 1/ np.sqrt(Norme_psi(weyl_func(I,k=k,A=1,sig=0.5,x0=-3),Nx,L))

psi0 =lambda x: weyl_func(x,k=k,A=C,sig=0.5,x0=-3)

V = lambda x,t: V_barriere(x,l,V0,d) 
V2 = lambda x,t: V_barriere(x,l/10,V0,d) 

K= np.sqrt(2*(V0-0.5*(k**2))); K2= K**2; k2=k**2
Trans = 4*(K2*k2) /( (K2+k2)**2 *K2* (l**2) +4*K2*k2)
print(Trans, V0, 0.5*K2)

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)
psi_2 = dynamics_fft_diss(psi0_fun=psi0,V_fun=V2, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)

### analyse des resultats
for i in range(0,Nt,int(np.floor(Nt/20))):
    t = T*i/Nt
    print("============================== ",t," ===========================")
    print(Concentration_de_masse(psi[:,i],Nx,L,-L,d)," masse avant l'interface")
    print(Concentration_de_masse(psi[:,i],Nx,L,d,d+l)," masse dans l'interface")
    print(Concentration_de_masse(psi[:,i],Nx,L,d+l,L)," masse apres l'interface")

    print("----------------------------------------------------------------")

    print(Concentration_de_masse(psi_2[:,i],Nx,L,-L,d)," masse avant l'interface")
    print(Concentration_de_masse(psi_2[:,i],Nx,L,d,d+l)," masse dans l'interface")
    print(Concentration_de_masse(psi_2[:,i],Nx,L,d+l,L)," masse apres l'interface")
    print("----------------------------------------------------------------")

    print("norme de psi sur [-L,L] = ", Norme_psi(psi[:,i],Nx,L))
    print("Energie de psi = ", Energie(psi[:,i],V,Nx,L,t))


In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="barriere de longueur l=2")
anime2 = plot_psi(psi_2,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V2, Time_length=T, subtitle="barriere de longueur l=0.2")
plt.close()
anime

In [ ]:
anime2

**Puit de potentiel**

Le probleme que va poser le puit de potentiel est de trouver un bon $V_0$ pour la simulation. Trop petit et l'on est confronter a un effet tunnel. Trop grand et l'on obtient des dispersion numeriques. 

On observe les memes effets de conservation de l'energie et de la probabilite que dans le cas de la barriere de potentiel. Il n'y a en effet pas de difference pour l'onde entre deux barriere de potentiel et un puit de potentiel. Meme theoriquement, on peut traiter le puit de potentiel comme une barriere epaisse car la fonction est $2L$ periodique. 

In [ ]:
## parametre simulation
Nx = 2000; Nt=500; L=10;  dilatation_temps =16; reduction=1
## parametre physique
k=8+0j; T=100; d=5; l=0.1; V0=100

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= 1/ np.sqrt(Norme_psi(weyl_func(I,k=k,A=1,sig=0.5,x0=-3),Nx,L))

psi0 =lambda x: weyl_func(x,k=k,A=C,sig=0.5,x0=-3)
V = lambda x,t: V_puit(x,V0,d)

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)

### analyse des resultats
for i in range(0,Nt,int(np.floor(Nt/5))):
    t = T*i/Nt
    print("============================== ",t," ===========================")
    print(Concentration_de_masse(psi[:,i],Nx,L,-d,d)," masse dans le puit")
    print(Concentration_de_masse(psi[:,i],Nx,L,-L,-d) + Concentration_de_masse(psi[:,i],Nx,L,L,d)," masse en dehors du puit")

    print("norme de psi sur [-L,L] = ", Norme_psi(psi[:,i],Nx,L))
    print("Energie de psi = ", Energie(psi[:,i],V,Nx,L,t))


N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt
    N.append(Norme_psi(psi[:,i],Nx,L,reduction=reduction)); E.append( Energie(psi[:,i],V,Nx,L,t,reduction=reduction))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b')
ax2.plot(time,E,'r')
plt.show()

In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="puit de potentiel")
plt.close()
anime

On voit que $V_0=100$ est suffisant, meme en temps long, pour peu que l'on ait une discretisation temporelle suffisamment elevee. 

Si l'on choisit desormais d'etudier les etat liees du puit de potentiel $\cos({\frac{n\pi}{d}x})$ et $\sin({\frac{n\pi}{d}x})$ en fonction de la parite de $n$.
On note qu'on trouve cette solution de la meme maniere que avec le cas de la barriere de potentiel.

theoriquement, pour que l'etat stationnaire plus haut soit un etat liee, c'est a dire que la fonction d'onde reste confinnee au puit de potentiel, il faut que 
\begin{equation}
    V(x)<V_c \qquad \qquad V_c = \frac{k^2\pi^2}{2d^2}
\end{equation}
On ne constate cependant rien de tel sur la simulation. 

On remarque des probleme lorsque l'on diminue la taille du puit en dessous de 0.5, discretiser le pas d'espace ou de temps n'a pas l'air d'ameliorer la solution. Une hypothese possible est qu'il s'agit de dissipation dans l'espace de Fourier, car $d^2$ est inversement proportionnel a la pulsation des solutions

In [ ]:
## parametre simulation
Nx = 2000; Nt=600; L=10;  dilatation_temps =100; reduction=1
## parametre physique
k=13+0j; T=100; d=5; V0=(k*np.pi)**2 /2*d**2

psi_0n =lambda x: np.cos((k)*np.pi/(2*d) *x)*(x>-d)*(x<d)
psi_1n =lambda x: np.sin((k+1)*np.pi/(2*d) *x)*(x>-d)*(x<d)
psi_2n =lambda x: np.sin((2)*np.pi/(2*d) *x)*(x>-d)*(x<d)

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= 1/ np.sqrt(Norme_psi(psi_0n(I),Nx,L))    ;C1= 1/ np.sqrt(Norme_psi(psi_1n(I),Nx,L))    ;C2= 1/ np.sqrt(Norme_psi(psi_2n(I),Nx,L))   

psi0 = lambda x :psi_0n(x)*C
psi01 = lambda x :psi_1n(x)*C1
psi02 = lambda x :psi_2n(x)*C2
V = lambda x,t: V_puit(x,V0,d)

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)
psi1 = dynamics_fft_diss(psi0_fun=psi01,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)
psi2 = dynamics_fft_diss(psi0_fun=psi02,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)

anime  = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="13e etat stationnaire")
anime1 = plot_psi(psi1,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="14e etat stationnaire")
anime2 = plot_psi(psi2,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="2e etat stationnaire")
plt.close()


In [ ]:

anime

In [ ]:
anime1

In [ ]:
anime2

**Potentiel quadratique**

On peut deja s'attendre a des probleme d'effet tunnel car le potentiel est bornee sur $[-L,L]$ par $V_0 L^2$
Une solution pour reduire les effets engendre par l'effet tunnel est d'augmenter $V_0$ de tel maniere a pouvoir considerer les etats liee aux systeme etant strictement inclue dans $[-L,L]$

On sait que pour un potentiel $V(x)= x^2$, les etat propre du systeme sont : $\Psi_n(x) = H_n(x) e^{-x^2/2}$ avec leurs energie associee $E_n= 2n+1$. Si l'on souhaite faire un changement de potentiel: $V(x) = V_0 x^2$, alors l'etat propre change aussi : $\Psi_n(x) = H_n(x \sqrt{\omega}) e^{-\frac{\omega x^2}{2}}$ avec $\omega = \sqrt{V_0}$

Un probleme qui vient assez naturellement est que si l'on augmente $V_0$ pour restreindre la solution et ne pas considerer les effets de periodicite, est que si on augmente trop $V_0$, on risque d'avoir une solution qui se disperse dans l'espace de Fourier. Cela peut avoir des effets nefaste sur la conservation de la probabilite ou de l'energie. 


Un probleme que l'on voit dans la simulation est que l'on perd facilement la conservation de l'energie. Plus precisement, pour les etat propre : 

- si l'on pose $L>\sqrt{2}$, l'energie oscille entre sa valeur initiale et une valeur superieur haute.
- si l'on pose $L<\sqrt{2}$, l'energie oscille entre sa valeur initiale et une valeur inferieur basse.
- si l'on pose $L=\sqrt{2}$, l'energie est conserve. 

On semble obtenir des effets similaire pour des etat mix mais qui n'ont pas l'air de dependre de $L$. 

Si on s'amuse a regarder la norme $H^1$, on voit aussi que la norme $H^1$ et l'energie ont l'air de suivre une oscillation de meme periode mais dephase de $\pi$.
Si l'on prend comme etat initiale une "fonction de weyl" avec une pulsation trop grande de tel sorte a ce qu'elle "sorte du puit quadratique", on peut observer des phenomene de resonnance pour la norme $H^1$ et l'energie. 

Il doit etre possible de calculer les temps auquel on a ces phenomene de resonnance en considerent le probleme posee sur $\mathbb{R}$ avec periodicite et une partie d'etat liee et une autre d'etat libre en fonction de leurs energie puis de calculer a quel instant il y a superposition de leurs fonctions d'onde. 

In [ ]:
## parametre simulation
Nx = 4000; Nt=600; L=np.sqrt(2);  dilatation_temps =100; reduction=1
## parametre physique
T=0.3; n=10; V0=1000; pulse = np.sqrt(V0)

H_n = sp.special.hermite(n)
psi_0n =lambda x: H_n(np.sqrt(pulse)*x)*np.exp(-pulse*(x**2)/2 )

# psi_0n =lambda x: weyl_func(x,k=1*L,A=1,sig=0.1)
# psi_0n =lambda x: weyl_func(x,k=10*L,A=1,sig=0.1)

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= 1/ np.sqrt(Norme_psi(psi_0n(I),Nx,L))

psi0 = lambda x :psi_0n(x)*C
V = lambda x,t: V_quad(x,0,V0)

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)

# ### analyse des resultats
# for i in range(0,Nt,int(np.floor(Nt/5))):
#     t = T*i/Nt
#     print("============================== ",t," ===========================")

#     print("norme de psi sur [-L,L] = ", Norme_psi(psi[:,i],Nx,L,reduction=reduction))
#     print("Energie de psi = ", Energie(psi[:,i],V,Nx,L,t,reduction=reduction))


N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt
    N.append(Norme_psi_Hs(psi[:,i],Nx,L,reduction=reduction,s=1)); E.append( Energie(psi[:,i],V,Nx,L,t,reduction=reduction))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b', label="norme $L^2$")
ax2.plot(time,E,'r', label="Energie")
ax.legend(loc="upper left"); ax2.legend(loc="upper right")
ax.set_xlabel("t")
fig.suptitle("conservation de la norme et de l'energie")
plt.show()

In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T,subtitle="10e etat stationnaire")
plt.close()
anime

**Potentiel regulier**

On regarde maintenant si il y a des differences majeurs entre une solution issue d'un potentiel discontinue et un potentiel plus regulier : 
- une des premiere remarque sur le potentiel regulier c'est que a memes parametre, le potentiel regulier prends plus de temps a calculer. Ce probleme est due au calcule des 2 splines
- on voit que si l'on compare les deux, sur un set de parametre qui induit de la perturbation numerique, alors on voit comme attendu que la solution resultant du probleme regularise est plus reguliere et possede moins de phenomene de perturbation numerique. De plus, plus $\varepsilon$ est grand, moins ces phenomenes apparaissent.
- on observe aussi que si $\varepsilon$ est petit, alors la solution regularise est solution du probleme initiale. Cela se prouve par theorie des perturbation en mettant le "raccord" comme perturbation du probleme initiale.

Le probleme est donc de choisir une $\varepsilon$ suffisamment grand pour qu'il n'y ait pas de perturbation numerique mais suffisament petit pour qu'il ne s'eloigne pas de la solution.

In [ ]:
## parametre simulation
Nx =4000; Nt=600; L=10;   dilatation_temps =20; reduction=10
## parametre physique
k=20+0j; T=10; d=0; l=0.2; V0=500; epsi=0.05


psi_0n =lambda x: weyl_func(I,k=k,A=1,sig=0.5,x0=-3)

## on normalise la fonction
I = np.linspace(-L,L,Nx)
C= reduction/ np.sqrt(Norme_psi(psi_0n(I),Nx,L))

psi0 = lambda x :psi_0n(x)*C
V = lambda x,t: V_barriere(x,l,V0,d) 
Vr = lambda x,t: Vr_barriere(x,l,V0,d,epsi=epsi) 
Vr2 = lambda x,t: Vr_barriere(x,l,V0,d,epsi=2*epsi) 

psi = dynamics_fft_diss(psi0_fun=psi0,V_fun=V, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)
psir = dynamics_fft_diss(psi0_fun=psi0,V_fun=Vr, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)
psir2 = dynamics_fft_diss(psi0_fun=psi0,V_fun=Vr2, L=L, Nx=Nx, T=T, Nt=Nt, dilatation_temps=dilatation_temps, reduction=reduction)


N=[]; E=[]; time = np.linspace(0,T,Nt)
Nr=[]; Er=[]
Nr2=[]; Er2=[]
for i in range(0,Nt):
    t= T*i/Nt
    N.append(Norme_psi(psi[:,i],Nx,L,reduction=reduction)); E.append( Energie(psi[:,i],V,Nx,L,t,reduction=reduction))
    Nr.append(Norme_psi(psir[:,i],Nx,L,reduction=reduction)); Er.append( Energie(psir[:,i],Vr,Nx,L,t,reduction=reduction))
    Nr2.append(Norme_psi(psir2[:,i],Nx,L,reduction=reduction)); Er2.append( Energie(psir2[:,i],Vr,Nx,L,t,reduction=reduction))

plt.ylabel("probabilite")
plt.xlabel("t")
plt.plot(time,N,'b', label="$epsilon$=0")
plt.plot(time,Nr,'k', label="$epsilon$=0.05")
plt.plot(time,Nr2,'r', label="$epsilon$=0.1")
plt.legend()
plt.show()

plt.ylabel("energie")
plt.xlabel("t")
plt.plot(time,E,'b', label="$epsilon$=0")
plt.plot(time,Er,'k', label="$epsilon$=0.05")
plt.plot(time,Er2,'r', label="$epsilon$=0.1")
plt.legend()
plt.show()

In [ ]:
anime = plot_psi(psi,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=V, Time_length=T, subtitle="barriere non regularise")
plt.close()
anime

In [ ]:
animer = plot_psi(psir,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=Vr, Time_length=T, subtitle="barriere regularise, $\epsilon =0.05$")
plt.close()
animer

In [ ]:
animer2 = plot_psi(psir2,L=L, duration=10, frames_per_second=20,show_potential=True,V_fun=Vr2, Time_length=T, subtitle="barriere regularise, $\epsilon =0.1$")
plt.close()
animer2

On peut regarder de la meme maniere le probleme du puit de potentiel regularise. Cependant on ne peut pas faire la meme etude des etat stationnaire car avec cette technique de regularisation, on change la taille du puit, ce qui va donc changer l'etat stationnaire. Il faudrait donc faire une etude de convergence des etats stationnaires perturbee vers les "vrai" etats stationnaire.

# Resume des problemes 1D

- **Potentiel nul** : au travers du potentiel nul, nous avons pu confirmer la validite du simulateur lorsqu'il n'y a pas de potentiel, nous avons en effet pu confirmer l'allure generale de la solution, la conservation de l'energie et de la norme $L^2$. Nous avons ensuite regarder qu'une solution stationnaire etait belle est bien stationnaire par ce modele. 

- **Barriere de potentiel** : avec la barriere de potentiel, nous avons pu confirmer que l'on garde la conservation de la norme $L^2$ lorsque l'on ajoute une interaction avec un potentiel. Nous avons aussi pu observer un effet semblable a l'effet tunnel sous quelques consideration numeriques.

- **Puit de potentiel** : avec le puit de potentiel, nous avons pu confirmer qu'un etat stationnaire theorique est aussi un etat stationnaire pour le simulateur, meme lorsque le potentiel est discontinue. L'interet particulier de ce potentiel est qu'on peut simplement le modelise theoriquement par un potentiel $2L$ periodique. Et si l'on choisi les parametres du probleme de tel sorte a negliger les interactions entre deux puit de potentiels, i.e on enleve l'effet tunnel et l'energie de l'onde est bien inferieur au potentiel, alors on retrouve le probleme du puit de potentiel theorique.

- **Potentiel quadratique** : avec le potentiel quadratique, nous avons pu confirmer le simulateur avec un nouveau test de solution stationnaire. On notera le probleme de la taille du domaine qui doit etre egale a $\sqrt{2}$ pour avoir avoir conservation de l'energie.

- **Regularisation des potentiels** : si l'on prend $\varepsilon$ petit, on trouve une solution proche du probleme initiale par perturbation. Si en revanche, nous le prenons plus grand, nous pouvons eviter les perturbations numerique. Il s'agit donc d'une histoire de dosage.

## Probleme 2D

Nous passerons tres vite sur quelques resultat de probleme 2D. Un probleme majeur de la simulation 2D est la taille memoire et le cout de calcul de la solution. Pour remediez a cela, nous utiliserons la methode de reduction et dilatation qui, bien qu'utilise lors des exemples de probleme 1D, a ete developpe en premier lieu pour contourner le probleme de stockage de solution 2D.

Un outil crucial qui n'a pas pu etre developpe a temps est une separation des degrees de libertee entre $N_x$ et $N_y$. Nous sommes forcees de prendre $N_x=N_y$, la ou des le premier exemple, il aurait ete utile de prendre $N_y$ petit et $N_x$ grands. Pour avoir acces a cela, il faudrait cependant refaire la fonction *dynamics_2D* et je n'en ai malheureusement pas le temps.

Nous utiliserons une color map 2D representant l'amplitude de la fonction d'onde pour visualiser la partie reel de la solution.

In [ ]:

from scipy.fft import fft2
from scipy.fft import ifft2
import time

from mpl_toolkits.mplot3d import Axes3D


def Kin_2D(Nx,Ny,Lx,Ly):
    K = np.zeros((Nx,Ny), dtype='complex')
    for i in range(0,Nx):
        K[i,:] = np.fft.fftfreq(Ny)

    for i in range(0,Ny):
        K[:,i] = K[:,i] + np.fft.fftfreq(Nx)
        
    K = K**2 * 0.25*(2*np.pi/(Lx) * 2*np.pi/(Ly))**2
    return K


def dynamics_2D( Lx, Ly, Nx, Ny, T, Nt, psi0_fun=(lambda x,y: np.exp(-(x*x+y*y)**2)), V_fun=(lambda x,y,t: 0), show=True, red_x=1, red_y=1,dilatation_temps=1):


    Kinetic = Kin_2D(Nx,Ny,Lx,Ly)
    I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

    Psi_temp =np.zeros((Nx,Ny,2), dtype="complex"); Phi_temp = np.zeros((Nx,Ny), dtype="complex")
    Psi_temp[:,:,0]=psi0_fun(I,J)
    dt = T/Nt
    
    Psi_T = np.zeros((int(Nx/red_x),int(Ny/red_y),Nt), dtype="complex")
    if(show) : Psi_T[:,:,0]= Projection_red_2D(Nx,Ny,Psi_temp[:,:,0],red_x=red_x, red_y=red_y)

    for i in range(1,dilatation_temps*Nt):
        ti = dt*i
        Phi_temp[:,:]= np.exp(-1j*Kinetic*dt)* fft2(np.exp(-1j*V_fun(I,J,ti)*dt) *Psi_temp[:,:,0])
        Psi_temp[:,:,1]= ifft2(Phi_temp[:,:]) 
        if(i%dilatation_temps==0) : 
            if(show) :  Psi_T[:,:,int(i/dilatation_temps)]  = Projection_red_2D(Nx,Ny,Psi_temp[:,:,1],red_x=red_x, red_y=red_y); 
            else :      Psi_T[:,:,int(i/dilatation_temps)]  = Projection_red_2D(Nx,Ny,Phi_temp[:,:]  ,red_x=red_x, red_y=red_y); 

        if(i%(dilatation_temps*50)==0) : print(i," pas de temps se sont ecoule sur ",dilatation_temps*Nt)
        Psi_temp[:,:,0] = Psi_temp[:,:,1]

    return Psi_T

In [ ]:
def plot_psi_2D(psi, duration=10, frames_per_second=20, Lx=10, Ly=10, Time_length=1, subtitle=" "):
    fig, ax = plt.subplots()
    
    Nt = psi[0,0,:].size
    frame_range = duration*frames_per_second; frame_step =int(Nt/frame_range)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    zmax = np.max(np.abs(psi))
    fig.suptitle(subtitle)
    

    ims=[]
    for i in range(0,frame_range,frame_step):
        ax.set_title(f"solution en t={ np.floor(100*t*Time_length)/100 }")
        im = ax.imshow(np.real(psi[:,:,i]),extent=[-Lx, Lx, -Ly, Ly], vmin=-zmax, vmax=zmax, animated=True)
        if i == 0:
            ax.imshow(np.real(psi[:,:,i]),extent=[-Lx, Lx, -Ly, Ly] )
        ims.append([im])
    fig.colorbar(im, ax=ax)
    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,repeat_delay=1000)
    return ani

def Norme_psi_2D(psi,Nx,Ny,red_x=1,red_y=1):
    Nx = int(Nx/red_x); Ny= int(Ny/red_y)
    C_psi = fft2(psi)
    norm = np.abs(C_psi)**2 
    return sum(sum(norm))


def Energie_2D(psi,V,Nx,Ny,Lx,Ly,t,red_x=1, red_y=1):
    Nx = int(Nx/red_x);  Ny= int(Ny/red_y)
    I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)
    C_psi = fft2(psi); V_psi= fft2(V(I,J,t)*psi)
    energ = 0.5*np.abs(C_psi)*(Kin_2D(Nx,Ny,Lx,Ly)) + np.abs(V_psi)
    energ *= red_x**2 * red_y**2
    return sum(sum(energ))

**Extension de probleme 1D**

On regarde si l'on obtiens les memes resultats en prenant une solution 1D et en lui rajoutant une dimension dont elle est independante

In [ ]:
##Potentiel nul
## parametre simulation
N=200; reduction=1
Nx = N; Ny= N; Nt=300; Lx=1; Ly=1; red_x=reduction; red_y=reduction; dilatation=20
## parametre physique
k=10+0j; T=10

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

psi0n =lambda x,y: eig_func(x,Lx*k,1) +0*y
## on normalise la fonction
C=1/np.sqrt(Norme_psi_2D(psi0n(I,J),Nx,Ny,red_x=red_x,red_y=red_y))
psi0 =lambda x,y: psi0n(x,y)*C
V = lambda x,y,t: V_0_2D(x,y) 

psi = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx,Ly=Ly, Nx=Nx,Ny=Ny, T=T, Nt=Nt)

anime_2D = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=10, frames_per_second=20, Time_length=T, subtitle="simulation 2D d'un potentiel nul")
plt.close()

# tranche de la simulation en (x,0) ou (0,y)

anime_x = plot_psi(psi[:,int(Ny/(2*red_y)),:],L=Lx,  duration=10, frames_per_second=20, Time_length=T,  subtitle="tranche de la simulation en (0,y)")
plt.close()
anime_y = plot_psi(psi[int(Nx/(2*red_x)),:,:],L=Ly,  duration=10, frames_per_second=20, Time_length=T,  subtitle="tranche de la simulation en (x,0)")
plt.close()

N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt * dilatation
    N.append(Norme_psi_2D(psi[:,:,i],Nx,Ny,red_x=red_x,red_y=red_y)); E.append(Energie_2D(psi[:,:,i],V,Nx,Ny,Lx,Ly,t,red_x=red_x,red_y=red_y))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b', label="norme $L^2$")
ax2.plot(time,E,'r', label="Energie")
ax.legend(loc="upper left"); ax2.legend(loc="upper right")
ax.set_xlabel("t")
fig.suptitle("conservation de la norme et de l'energie")
plt.show()

In [ ]:
anime_x

In [ ]:
anime_y

In [ ]:
anime_2D

In [ ]:
##Bariere de potentiel

## parametre simulation
N=300; reduction=1
Nx = N; Ny= N; Nt=300; Lx=2; Ly=1; red_x=reduction; red_y=reduction; dilatation=10
## parametre physique
k=5+0j; T=20; V0=1; d=0;l=0.05

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

psi0n= lambda x,y:  weyl_func(x,k,1,x0=-0.5,sig=0.1)+0*y
## on normalise la fonction
C=1/np.sqrt(Norme_psi_2D(psi0n(I,J),Nx,Ny,red_x=red_x,red_y=red_y))
psi0 =lambda x,y: psi0n(x,y)*C

V = lambda x,y,t: V_barriere(x,l,V0,d) + y*0
Vx = lambda y,t : V(0,y,t) ; Vy = lambda x,t: V(x,0,t)

psi = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx, Ly=Ly, Nx=Nx,Ny=Ny, T=T, Nt=Nt,red_x=red_x, red_y=red_y, dilatation_temps=dilatation)

anime_2D = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=10, frames_per_second=20, Time_length=T, subtitle="simulation en 2d de la barriere de potentiel")
plt.close()

# tranche de la simulation en (x,0) ou (0,y)
anime_x = plot_psi(psi[:,int(Ny/(2*red_y)),:],L=Lx,  duration=10, frames_per_second=20,show_potential=True,V_fun=Vx, Time_length=T,  subtitle="tranche de la simulation en (0,y)")
plt.close()
anime_y = plot_psi(psi[int(Nx/(2*red_x)),:,:],L=Ly,  duration=10, frames_per_second=20,show_potential=True,V_fun=Vy, Time_length=T,  subtitle="tranche de la simulation en (x,0)")
plt.close()

N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt * dilatation
    N.append(Norme_psi_2D(psi[:,:,i],Nx,Ny,red_x=red_x,red_y=red_y)); E.append(Energie_2D(psi[:,:,i],V,Nx,Ny,Lx,Ly,t,red_x=red_x,red_y=red_y))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b', label="norme $L^2$")
ax2.plot(time,E,'r', label="Energie")
ax.legend(loc="upper left"); ax2.legend(loc="upper right")
ax.set_xlabel("t")
fig.suptitle("conservation de la norme et de l'energie")
plt.show()

In [ ]:
anime_y

In [ ]:
anime_2D

In [ ]:
anime_x

In [ ]:
##Puit de potentiel

## parametre simulation
N=200; reduction=1
Nx = N; Ny= N; Nt=300; Lx=2; Ly=1; red_x=reduction; red_y=reduction; dilatation=5
## parametre physique
k=5; T=20; V0=1; d=1

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

psi0n =lambda x,y: np.cos((k)*np.pi/(2*d) *x)*(x>-d)*(x<d) +0*y

## on normalise la fonction
C=1/np.sqrt(Norme_psi_2D(psi0n(I,J),Nx,Ny,red_x=red_x,red_y=red_y))
psi0 =lambda x,y: psi0n(x,y)*C

V  = lambda x,y,t: V_puit(x,V0,d) + y*0
Vx = lambda y,t  : V(0,y,t) 
Vy = lambda x,t  : V(x,0,t)

psi = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx, Ly=Ly, Nx=Nx,Ny=Ny, T=T, Nt=Nt,red_x=red_x, red_y=red_y, dilatation_temps=dilatation)

anime_2D = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=10, frames_per_second=20, Time_length=T, subtitle="simulation en 2D du puit de potentiel")
plt.close()

# tranche de la simulation en (x,0) ou (0,y)
anime_x = plot_psi(psi[:,int(Ny/(2*red_y)),:],L=Lx,  duration=10, frames_per_second=20,show_potential=True,V_fun=Vx, Time_length=T,  subtitle="tranche de la simulation en (0,y)")
plt.close()
anime_y = plot_psi(psi[int(Nx/(2*red_x)),:,:],L=Ly,  duration=10, frames_per_second=20,show_potential=True,V_fun=Vy, Time_length=T,  subtitle="tranche de la simulation en (x,0)")
plt.close()

N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt * dilatation
    N.append(Norme_psi_2D(psi[:,:,i],Nx,Ny,red_x=red_x,red_y=red_y)); E.append(Energie_2D(psi[:,:,i],V,Nx,Ny,Lx,Ly,t,red_x=red_x,red_y=red_y))

fig, ax = plt.subplots()

ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b', label="norme $L^2$")
ax2.plot(time,E,'r', label="Energie")
ax.legend(loc="upper left"); ax2.legend(loc="upper right")
ax.set_xlabel("t")
fig.suptitle("conservation de la norme et de l'energie")
plt.show()

In [ ]:
anime_y

In [ ]:
anime_2D

In [ ]:
anime_x

On peut donc constater que si l'on prends un potentiel et une solution initiale independant de $y$ , alors il n'y a pas de changement entre la simulation 1D et 2D. On peut faire les memes tests sur l'independance en $x$.

On constate des pertes d'energie qui peuvent etre attribue a la precision deplorable des simulations. Nous ne pouvons donc pas esperer utiliser l'energie comme un indicateur utile dans cette partie.

Si l'on regarde maintenant des solution initiales ou des potentiels qui dependent de $y$, nous pouvons regarder si l'on garde certaines proprietes.

In [ ]:
## onde gaussienne dirigee par un vecteur d'onde

## parametre simulation
N=200; reduction = 1
Nx = N; Ny= N; Nt=300; Lx=1; Ly=1; red_x=reduction; red_y=reduction; dilatation=5
## parametre physique
kx=0; ky=0; T=20; V0=1; d=0;l=0.05; sig=0.1;     x0=0;y0=0

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

psi0n= lambda x,y:  np.exp(-((x-x0)**2 + (y-y0)**2)/(2*sig**2)) *np.exp(1j *2*np.pi*kx*(x-x0)) *np.exp(1j*2*np.pi*ky*(y-y0))
## on normalise la fonction
C=1/np.sqrt(Norme_psi_2D(psi0n(I,J),Nx,Ny,red_x=red_x,red_y=red_y))
psi0 =lambda x,y: psi0n(x,y)*C

# V = lambda x,y,t: V_barriere(x,l,V0,d) + y*0
V = lambda x,y,t: 0*x+0*y
Vx = lambda y,t : V(0,y,t) ; Vy = lambda x,t: V(x,0,t)

psi = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx, Ly=Ly, Nx=Nx,Ny=Ny, T=T, Nt=Nt,red_x=red_x, red_y=red_y, dilatation_temps=dilatation)

anime_2D = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=10, frames_per_second=20, Time_length=T, subtitle="Erreur de simulation : anisotropie")
plt.close()


# N=[]; E=[]; time = np.linspace(0,T,Nt)
# for i in range(0,Nt):
#     t= T*i/Nt * dilatation
#     N.append(Norme_psi_2D(psi[:,:,i],Nx,Ny,red_x=red_x,red_y=red_y)); E.append(Energie_2D(psi[:,:,i],V,Nx,Ny,Lx,Ly,t,red_x=red_x,red_y=red_y))

# fig, ax = plt.subplots()
# ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
# ax.plot(time,N,'b')
# ax2.plot(time,E,'r')
# plt.show()

In [ ]:
anime_2D

On constate ici que la fonction d'onde ne se propage pas de maniere egale dans le milieu. Le milieu est donc anisotrope. L'hypothese la plus probable est que la fonction d'evolution *dynamics_2D* soit biaise par la direction (-1,1). Par soucis de manque de temps, nous ne pourrons malheureusement pas etudiee les fonctions d'onde dependant de $x$ et de $y$. 

# Une derniere folie

supposons qu'on ait un simulateur 2d qui fonctionne pour un potentiel et une condition initiale assez gentil. Le point culminant de ce rapport serait alors la reproduction de l'experience des fentes de Young. On sait que la simulation ne fonctionnera pas car comme expliquee juste au dessus, la simulation 2d est biaise vers une direction et l'experience des fentes de Young depend de maniere explicite de l'isotropie du milieu mais essayons d'en faire abstraction pendant 5s. 

In [ ]:
##Fentes de Young

## parametre simulation
N=300; reduction=1
Nx = N; Ny= N; Nt=300; Lx=5; Ly=1; red_x=reduction; red_y=reduction; dilatation=10
## parametre physique
k=5+0j; T=20; V0=100; d=0; epaisseur=0.2; ouverture=0.05; ecart=0.5

I = np.linspace(-Lx,Lx,Nx); J = np.linspace(-Ly,Ly,Ny).reshape(-1,1)

psi0n= lambda x,y:  weyl_func(x,k,1,x0=-0.5,sig=0.1)+0*y
## on normalise la fonction
C=1/np.sqrt(Norme_psi_2D(psi0n(I,J),Nx,Ny,red_x=red_x,red_y=red_y))
psi0 =lambda x,y: psi0n(x,y)*C

V = lambda x,y,t: V_barriere(x,epaisseur,V0,d)*((y<-ecart-ouverture) + (y>-ecart)*(y<ecart) + (y>ecart+ouverture))
Vx = lambda y,t : V(0,y,t) ; Vy = lambda x,t: V(x,0,t)


psi = dynamics_2D(psi0_fun=psi0,V_fun=V, Lx=Lx, Ly=Ly, Nx=Nx,Ny=Ny, T=T, Nt=Nt,red_x=red_x, red_y=red_y, dilatation_temps=dilatation)

anime_2D = plot_psi_2D(psi,Lx=Lx,   Ly=Ly, duration=10, frames_per_second=20,  Time_length=T, subtitle="simulation des fentes de")
plt.close()

# tranche de la simulation en (L,y)

anime_x = plot_psi(psi[:,-1,:],L=Lx,  duration=10, frames_per_second=20,show_potential=True,V_fun=Vx, Time_length=T,  subtitle="tranche de la simulation en (L,y)")
plt.close()

N=[]; E=[]; time = np.linspace(0,T,Nt)
for i in range(0,Nt):
    t= T*i/Nt * dilatation
    N.append(Norme_psi_2D(psi[:,:,i],Nx,Ny,red_x=red_x,red_y=red_y)); E.append(Energie_2D(psi[:,:,i],V,Nx,Ny,Lx,Ly,t,red_x=red_x,red_y=red_y))

fig, ax = plt.subplots()
ax2= ax.twinx(); ax.set_ylabel("probabilite"); ax2.set_ylabel("energie")
ax.plot(time,N,'b')
ax2.plot(time,E,'r')
plt.show()

In [ ]:
anime_2D

In [ ]:
anime_y

On remarque deux cretes en $(L,y)$, symbole de l'anisotropie de la fonction d'evolution. Si la fonction d'evolution 2D fonctionnait parfaitement, alors nous aurions eu des phenomenes de resonnances similaire a l'experience de Young.

## Conclusion

Nous avons pu montrer au travers de ce rapport des resultats simple de simulation comme la conservation de la norme $L^2$ et de l'energie. Nous avons aussi montrer la coherence de la simulation avec les modeles mathematiques au travers des etat propre de differents potentiels. Nous avons ensuite essayer d'etendre ces resultats aux simulations en $2d$.

Nous avons pu montrer que sous certaines contraintes, les simulations en $2d$ etaient valides. Cependant ces contraintes sont similaire a "pouvoir ecrire le probleme comme probleme 1d" et les coûts de calculs de probleme en $2d$ sont tels qu'il n'y a aucun avantage a resoudre ce probleme en $2d$ lorsqu'on peut le faire en $1d$. 

Enfin, si l'on essayer de s'affranchir de ses contraintes, le simulateur $2d$ ne suit plus et nous renvoie une solution d'un milieu anisotrope. Le fait qu'on puisse simuler un milieu anisotrope sans avoir a toucher au potentiel est une ouverture interessante, mais si l'on souhaite traiter le cas d'un milieu isotrope et homogene, cela ne nous sert peu.

Une conclusion plus satisfaisante aurait pu etre atteinte, helas je n'ai pas le temps de corriger le simulateur $2d$.